In [19]:
import numpy as np 
import scipy.integrate as integ
from IPython.display import Math
import matplotlib.pyplot as plt


\begin{equation}
f'(x)  = -3x
\end{equation}



In [20]:
def test(x, t):
    return -3*x[0]

In [21]:
#initial conditions
x0 = [1]
t = np.linspace(0, 4, 9)

In [22]:
sol = integ.odeint(test, x0, t)

In [23]:
fig1 = plt.figure()
ax1 = fig1.add_subplot(111)
ax1.plot(t, sol)
plt.show()

\begin{equation}
mx''(t) + cx'(t) + kx(t) = 0\\
\downarrow\\
x_1'(t) = x_2(t)\\
x_2'(t) = -\frac{c}{m}\ x_2(t) - \frac{k}{m}\ x_1(t)
\end{equation}


In [24]:
def vib1(x, t):
    m = 3
    c = 1
    k = 2
    
    return [x[1], -(c/m)*x[1]-(k/m)*x[0]]

\begin{equation}
x(t) = 0\\
x'(t) = 0.25\\
0 \leq t \leq 20
\end{equation}

In [25]:
x0 = [0, 0.25]
t = np.linspace(0, 50, 1000)

In [26]:
solvib = integ.odeint(vib1, x0, t)

In [27]:
fig2 = plt.figure()
ax2 = fig2.add_subplot(111)
ax2.plot(t, solvib)
ax2.set_xlabel('Time')
ax2.set_ylabel('Displacement\nVelocity')
leg2 = ax2.legend(['Displacement','Velocity'])
leg2.get_frame().set_linewidth(0.0)
plt.show()

Pêndulo rotacionando 
\begin{equation}
\theta''(t) + \bigg(\frac{3g}{2l} - {\Omega^2}cos(\theta(t))\bigg) sin(\theta(t)) = 0\\
\downarrow\\
\theta_1'(t) = \theta_2(t)\\
\theta_2'(t) = -\bigg(\frac{3g}{2l} - {\Omega^2}cos(\theta_1(t))\bigg) sin(\theta_1(t))
\end{equation}

In [28]:
g = 9.81
l = 1
omega = 50

def pendulum(o, t):

    
    return [o[1], -((3*g/(2*l)-((omega**2)*np.cos(o[0]))))*np.sin(o[0])]

In [29]:
teta0 = np.arccos(3*g/(2*l*omega**2))
o0 = [teta0+(20*np.pi/180), 0]

In [30]:
solpend = integ.odeint(pendulum, o0, t)

In [31]:
(3*g/(2*l*omega**2))

0.005886

\begin{equation}
\theta(t) = Asin(\omega_nt+\phi)\\
\downarrow\\
A = \sqrt{\frac{w_n^2 x_0^2 + v_0^2}{w_n^2}}\\
\phi = arctan\bigg(\frac{w_n x_0}{v_0}\bigg)
\end{equation}

In [32]:
#Solução analítica
teta = teta0+(20*np.pi/180)
o_0 = [teta-teta0,0]
wn = ((-3*g*np.cos(teta0)/(2*l))+omega**2)**0.5
#wn = ((-3*g/(2*l))+(omega**2))**0.5
A = (((wn**2)*(o_0[0]**2)+(o_0[1]**2))/(wn**2))**0.5
if o_0[1] == 0:
    phi = np.pi/2
else:
    phi = np.arctan(wn*0/o_0[1])
oa = A*np.sin(wn*t + phi)

In [33]:
solpendan = teta0+A*np.sin(wn*t+phi)

In [34]:
fig3 = plt.figure()
ax3 = fig3.add_subplot(111)
ax3.plot(t, solpendan)
ax3.plot(t, solpend[:,0])
ax3.set_xlabel('Time')
ax3.set_ylabel('Displacement\nVelocity')
ax3.set_yticks([0., np.pi/4, np.pi/2, 3*np.pi/4, np.pi])
ax3.set_yticklabels(['0','45','90','135','180'])
leg3 = ax3.legend(['Linear','Non linear'])
leg3.get_frame().set_linewidth(0.0)
plt.show()

In [41]:
class VibSystemForced(object):
    
    def __init__(self, m, c, k, t, x0, F, w_f, phi_f=0):
        self.m = m
        self.c = c
        self.k = k
        self.t = t
        self.x0 = x0
        self.F = F
        self.w_f = w_f
        self.phi_f = phi_f       
 
    #--------------------------------------------RESPOSTA DO SISTEMA--------------------------------------------
    
    def response(self, x, t, force_type):
        m = self.m
        c = self.c
        k = self.k      
        F = self.F
        w_f = self.w_f #frequência de forçamento
        phi_f = self.phi_f 
        if force_type == 'cos':
            force = F*np.cos(w_f*t + phi_f)
        
        elif force_type == 'sen':
            force = F*np.cos(w_f*t + phi_f)
            
        elif force_type == 'impulse':
            force = F
        
        return [x[1], (force/m) - (c/m)*x[1] -(k/m)*x[0]]        
    
    
    #--------------------------------------------FORÇAMENTOS POSSÍVEIS--------------------------------------------
    
    def force_cos(self): # integra a função response para o forçamento escolhido (cos)
        self.t = t              
        force_type = 'cos'
        return integ.odeint(self.response, self.x0, self.t, args=(force_type,))  
    
    
    #--------------------------------------------PARAMETROS DO SISTEMA--------------------------------------------

    def wn(self):
        return (self.k/self.m)**0.5
    
    def damp_ratio(self):
        return self.c/(2*(self.k*self.m)**0.5)
    
    def log_dec(self):
        return 2*np.pi*self.damp_ratio()/((1-self.damp_ratio()**2)**0.5)
    
    def wd(self):
        return self.wn()*(1-self.damp_ratio()**2)**0.5
    
    def period(self):
        return 1/self.wd()
    
    def amp(self):
        x0 = self.x0[0]
        v0 = self.x0[1]
        zeta = self.damp_ratio()
        wn = self.wn()
        wd = self.wd()
        
        return (((v0+zeta*x0*wn)**2+(x0*wd)**2)**0.5)/wd
    
    def phi(self):
        x0 = self.x0[0]
        v0 = self.x0[1]
        zeta = self.damp_ratio()
        wn = self.wn()
        wd = self.wd()
    
        return np.arctan((x0*wd)/(v0+zeta*wn*x0))
    
    #--------------------------------------------------GRÁFICOS----------------------------------------------------
    
    def plot_disp(self, force_type):
        dic = {'force_cos':self.force_cos()}
        fig1 = plt.figure()
        ax1 = fig1.add_subplot(111)
        ax1.plot(self.t, dic[force_type][:,0])
        ax1.set_xlabel('Time')
        ax1.set_ylabel('Displacement')
        leg1 = ax1.legend(['Displacement'])
        leg1.get_frame().set_linewidth(0.0)
        plt.show()
        
    def plot_vel(self):
        fig2 = plt.figure()
        ax2 = fig2.add_subplot(111)
        ax2.plot(self.t, self.solvib2()[:,1])
        ax2.set_xlabel('Time')
        ax2.set_ylabel('Velocity')
        leg2 = ax2.legend(['Velocity'])
        leg2.get_frame().set_linewidth(0.0)
        plt.show()
        
    def plot_disp_vel(self):
        fig3 = plt.figure()
        ax3 = fig3.add_subplot(111)
        ax3.plot(self.t, self.solvib2())
        ax3.set_xlabel('Time')
        ax3.set_ylabel('Displacement\nVelocity')
        leg3 = ax3.legend(['Displacement','Velocity'])
        leg3.get_frame().set_linewidth(0.0)
        plt.show()
        
        

In [42]:
vibex1 = VibSystemForced(100, 200, 2000, t, [0.0, 0.1], 150, 10)

In [43]:
vibex1.plot_disp('force_cos')

In [ ]:
"""    def force_cos(self, t):
        F = self.F
        w_f = self.w_f #frequência de forçamento
        phi_f = self.phi_f
        
        return F*np.cos(w_f*t+phi_f)
    
    def impulse(self):
        F = self.F
        if t = 0:
            return F
        else:
            return 0
        
    
    #def impulse_response(self, F0):
        
        
    def response(self, x, t):
        m = self.m
        c = self.c
        k = self.k
        
        force = self.force_cos(t)
        
        return [x[1], (force/m)-(c/m)*x[1]-(k/m)*x[0]]             
    
    def solvib2(self):        
        return integ.odeint(self.response, self.x0, self.t)"""

In [ ]:
force_cos = 0
d={'a':force_cos}

In [ ]:
b = 'a'


In [ ]:
d[b]